In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import matplotlib.pyplot as plt
from keras_tqdm import TQDMNotebookCallback
from npm3dGenerator_tree import NPM3DGenerator, NPM3DGenerator_full
from models import build_point_net, save_model, load_model
import utils_training as u_t

Using TensorFlow backend.


In [20]:
training_dataset = NPM3DGenerator(paths_to_keep = [1, 2])
validation_dataset = NPM3DGenerator(paths_to_keep = [0])

not path is None ../Benchmark_MVA/training\MiniLille2.ply
../Benchmark_MVA/training\MiniLille2_normals.npy
else ../Benchmark_MVA/training\MiniLille2_normals.npy
not path is None ../Benchmark_MVA/training\MiniParis1.ply
../Benchmark_MVA/training\MiniParis1_normals.npy
else ../Benchmark_MVA/training\MiniParis1_normals.npy
[2500428 4159318]


not path is None ../Benchmark_MVA/training\MiniLille1.ply
../Benchmark_MVA/training\MiniLille1_normals.npy
else ../Benchmark_MVA/training\MiniLille1_normals.npy
[1901853]


In [19]:
validation_dataset.sample_point_cloud()

(1901853, 3) (4096,) [[ 20.04634094   4.18713379   7.24810791]
 [ -2.85209656   0.46478271  -2.49810028]
 [ 47.64381409  20.65917969  -0.65337753]
 ...
 [-30.23158264 -41.21975708  -3.24214935]
 [-30.8250885  -41.2331543   -3.25870514]
 [-29.94126892 -41.43814087  -3.24028778]]


(array([[-0.52511305,  0.40101143, -0.02925736, ...,  0.9963681 ,
          0.00460867,  0.89256391],
        [-0.49722476,  0.39711879, -0.04967362, ...,  0.99646578,
          0.00442049,  0.8756687 ],
        [-0.49094166,  0.36456334, -0.03102357, ...,  0.99641793,
          0.00452152,  0.86745675],
        ...,
        [-0.74322433, -2.35208709,  0.18579238, ...,  0.99474012,
          0.00680357,  0.69789169],
        [-0.43563361, -2.36063789,  0.17688888, ...,  0.99443808,
          0.0074672 ,  0.81054254],
        [-1.02343271, -2.31445436,  0.20598739, ...,  0.99642874,
          0.00654478,  0.70060313]]), array([[1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        ...,
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0.]], dtype=float32))

In [3]:
miou_metric = u_t.mIoU(training_dataset.n_classes)

model = build_point_net(input_shape = (training_dataset.n_points, training_dataset.n_channels), output_shape = training_dataset.n_classes)
#model = load_model("trained_{}_{}_{}".format(test_dataset.n_points, test_dataset.n_channels, test_dataset.n_classes))
model.compile("adam", u_t.my_categorical_crossentropy, metrics = ["accuracy", miou_metric.miou])
#model.summary()

In [10]:
n_epochs = 10
history = model.fit_generator(training_dataset, epochs = n_epochs,
                              validation_data = validation_dataset,
                              class_weight = training_dataset.class_weight,
                              verbose = 0, callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
#save_model(model, "trained_{}_{}_{}".format(training_dataset.n_points, training_dataset.n_channels, training_dataset.n_classes))
u_t.plot_history(history)

10



(1901853, 3)(4159318, 3)  (4096,)(4096,)

0
1
(1901853, 3) (4096,)(4159318, 3)
 (4096,)


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
test_dataset = NPM3DGenerator(input_dir = "../Benchmark_MVA/test", train = False)
#model = load_model("trained_{}_{}_{}".format(test_dataset.n_points, test_dataset.n_channels, test_dataset.n_classes))
predictions = test_dataset.predict_point_cloud(model)
plt.hist(np.argmax(predictions, axis = -1))

In [ ]:
np.savetxt('MiniDijon9.txt', np.argmax(predictions, axis = -1) + 1, fmt='%d')